<a href="https://colab.research.google.com/github/samibahig/RecoverProject/blob/main/RidgeClassifierProteomique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import sklearn
import os
plt.style.use('ggplot')
#print('\nMETADATA :')
data_path = ''
metadata_filename = data_path + '/content/metadata.csv'
meta_df = pd.read_csv(metadata_filename)
#print(meta_df.columns)
meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
#print(meta_df.columns)
#print('available metadata :', list(meta_df))
meta_idx = meta_df['ID'].to_list()
meta_label = meta_df['symptoms'].to_list()
#print('------------------')
#print(list(zip(meta_idx, meta_label)))
#print('------------------')
meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}
data_path = ''

#DF1 : proteomics
#print('\nPROEOMICS DATA :')
proteomics_data_filename = '/content/proteomics.csv'

dim_df = pd.read_csv(proteomics_data_filename, nrows=1)
#print('--------')
#print(dim_df)
#print('--------')
dim = len(list(dim_df))
#print(dim)
#print('------------')
print('# of columns in source csv file :', dim)
all_cols = [i for i in range(dim)]

print('--------')
feat_cols = all_cols[1:-4]
print(feat_cols)
print('--------')
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(proteomics_data_filename, skiprows=6, index_col=0, skipfooter=4, usecols=[0], engine='python')
idx = list(idx_df.index.values)
print('# of idx : ', len(idx))
print('first id :', idx[0])
print('last id :', idx[-1])

df1 = pd.read_csv(proteomics_data_filename, skiprows=6, dtype=np.float32, skipfooter=4, usecols=feat_cols, engine='python')
assert df1.shape[0] == len(idx)
assert df1.shape[1] == len(features)

df1['idx'] = idx
df1.set_index('idx', inplace=True)
df1.columns = features
print('# of Nan values :', df1.isna().sum().sum())

#clean data of samples that are not in metadata :
idx = df1.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df1['label'] = y
df1 = df1[df1.label != 'to_remove']

#create X and y matrices for ML :
y = list(df1['label'])
del df1['label']
print('---------')
print(df1)
print('---------')
X = df1.to_numpy()
print('proteomics data :')
print('# of samples : ', df1.shape[0])
print('# of features : ', df1.shape[1])
print('labels:', list(dict.fromkeys(y)))


## save X and y in pickles if you want :
##data_name = 'recover_multiomics_'
##feat_dict = {k: str(v) for k, v in zip(range(len(list(df))), list(df))}
##with open(data_path + data_name + 'feat_dict', 'wb') as fo:
##    pkl.dump(feat_dict, fo)
##with open(data_path + data_name + 'X', 'wb') as fo:df
##            pkl.dump(X, fo)
##with open(data_path + data_name + 'y', 'wb') as fo:
##            pkl.dump(y, fo)

# of columns in source csv file : 189
--------
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]
--------
# of features :  184
first feature : Q96SB3
last feature : P09603
# of idx :  104
first id : 5-139
last id : 3-043
# of Nan values 

In [21]:
### Ici entrainement du modèle Ridge avec alpha=1.0
from sklearn.datasets import make_regression
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
import numpy as np
clf = RidgeClassifier(alpha=1.0)
clf.fit(X_train, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [22]:
### Ici les prédictions sont calculées  sauf que je n'arrive pas à calculer l'accuracy?  As tu des hints? Merci 
from sklearn.metrics import accuracy_score
from sklearn import metrics
predictions = clf.predict(X_test)
predictions
print("Test Accuracy:", metrics.accuracy_score(y_test, predictions))

Test Accuracy: 0.45


In [23]:
clf.get_params(deep=True)

{'alpha': 1.0,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [25]:
###  Boostraping , Pour Thibault, tu peux essayer ici, j'obtiens toujours 1.0, c'est trop parfait, non ?
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=60

## bootstrapping
from sklearn import metrics
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)
    clf = RidgeClassifier(alpha=1.0)
    clf.fit(X_train, y_train)
    ########################################################
    predictions = clf.predict(X_test)
    Accuracy=metrics.accuracy_score(y_test, predictions)
    AccuracyValues.append(np.round(Accuracy))

# Result of all bootstrapping trials
print(y_test, predictions)
print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues))

[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0] [0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1 1]
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Final average accuracy 0.4666666666666667


In [26]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
params_Ridge = {'alpha': [0.1,1.0,10.0]}
alpha = (0.1, 1.0, 10.0)
random_grid = {'alpha': alpha}
pprint(random_grid)

{'alpha': (0.1, 1.0, 10.0)}


In [28]:
clf.get_params().keys()

dict_keys(['alpha', 'class_weight', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'random_state', 'solver', 'tol'])

In [29]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import RidgeCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestClassifier()
result = RidgeClassifier()
#result.score(X_test)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
result = RandomizedSearchCV(estimator = result, param_distributions = random_grid, n_iter = 200, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
result.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=200. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.9s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                             copy_X=True, fit_intercept=True,
                                             max_iter=None, normalize=False,
                                             random_state=None, solver='auto',
                                             tol=0.001),
                   iid='deprecated', n_iter=200, n_jobs=-1,
                   param_distributions={'alpha': (0.1, 1.0, 10.0)},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [30]:
result.best_params_

{'alpha': 1.0}

In [31]:
### Nouvelle instantiation avec les paramètres trouvés en Cross-Validation
result = RidgeClassifier(alpha = 1.0)

In [32]:
### entraînement du modèle avec les paramètres trouvés en Cross-Validation
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
result.fit(X_train, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [33]:
### Calcul de y_test et de l'accuracy 
from sklearn.metrics import accuracy_score
from sklearn import metrics
predictions = result.predict(X_test)
print("Test Accuracy:", metrics.accuracy_score(y_test, predictions))

Test Accuracy: 0.45


In [34]:
### Vérification avec f1_score
from sklearn.metrics import f1_score
f1_score(y_test, predictions)

0.42105263157894735

In [35]:
### Boostraping 60 fois avec Paramètres obtenus en Cross-Validation
from sklearn.model_selection import train_test_split 
from sklearn import metrics
#### Bootstrapping ####
########################################################
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=60
## Performing bootstrapping
from sklearn import metrics
from sklearn.model_selection import train_test_split 
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)
    result = RidgeClassifier(alpha = 0.1)
    result.fit(X_train, y_train)
    ########################################################
    prediction = result.predict(X_test)
    Accuracy=metrics.accuracy_score(y_test, prediction)
    print(Accuracy)
    AccuracyValues.append((Accuracy))
    #print(Accuracy)
    print(AccuracyValues)
    
    #Creating the model on Training Data
    #DTree=RegModel.fit(X_train,y_train)
    #prediction=DTree.predict(X_test)
 
    #Measuring accuracy on Testing Data
#Accuracy=100- (np.mean(np.abs((y_test - prediction) / y_test)) * 100)
    
    # Storing accuracy values
#AccuracyValues.append(np.round(Accuracy))
    
################################################
# Result of all bootstrapping trials
print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues), ' std ', np.std(AccuracyValues))
#print("Test Accuracy:", metrics.accuracy_score(y_test, y_final ))

0.45
[0.45]
0.55
[0.45, 0.55]
0.35
[0.45, 0.55, 0.35]
0.4
[0.45, 0.55, 0.35, 0.4]
0.35
[0.45, 0.55, 0.35, 0.4, 0.35]
0.65
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65]
0.45
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45]
0.35
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35]
0.5
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5]
0.5
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5]
0.4
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4]
0.7
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7]
0.6
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7, 0.6]
0.55
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7, 0.6, 0.55]
0.5
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7, 0.6, 0.55, 0.5]
0.65
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7, 0.6, 0.55, 0.5, 0.65]
0.3
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 0.45, 0.35, 0.5, 0.5, 0.4, 0.7, 0.6, 0.55, 0.5, 0.65, 0.3]
0.65
[0.45, 0.55, 0.35, 0.4, 0.35, 0.65, 